该内容是对simple_vit中内容的讲解，主要分为：
- 代码
    - attention
    - transformer
    - vit
- 训练
    - train
- 测试
    - test
 
> 与simple_vit中相重合的部分不会再说明

In [ ]:
import torch
from torch import nn
from einops import rearrange

# 一、代码

## 1 attention

```python
project_out = not (heads == 1 and dim_head == dim)
```

如果是 单头注意力，并且dim_head = dim（即注意力输出和输入维度相同），那么就不需要额外线性映射，直接输出，也就是后面的将to_out 设为 Identity()
> 这里其实没太大区别，主要是后面加了个Dropout


```python
self.dropout = nn.Dropout(dropout)
...
self.to_out = nn.Sequential(
    nn.Linear(inner_dim, dim),
    nn.Dropout(dropout)
) if project_out else nn.Identity()
```
主要是两个地方，多加了两个Dropout，原因还是那么几个：
- 在深层结构中，避免过拟合（核心目的）
- 与 Transformer 论文一致

In [ ]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5
        self.norm = nn.LayerNorm(dim)
        self.attend = nn.Softmax(dim = -1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


## 2 transformer

### 2.1 FeedForward

```python
self.net = nn.Sequential(
    nn.LayerNorm(dim),
    nn.Linear(dim, hidden_dim),
    nn.GELU(),
    nn.Dropout(dropout),
    nn.Linear(hidden_dim, dim),
    nn.Dropout(dropout)
)
```
相比于simple，多了两个Dropout，原因与前面相同。
其余部分与simple相同。

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)


## 3 ViT

有了simple_vit的基础，ViT中我就不再将Patch Embedding部分分出去了，因为ViT中没有用到单独的位置编码方式，所以整体理解起来反而会更简单。

### 3.1 ViT

image_size: 图像尺寸，可以是整数或 (高, 宽)。               
patch_size: patch 的大小，同样可以是整数或 (高, 宽)。                
num_classes: 分类类别数量。                
dim: transformer 中 token 的特征维度。              
depth: Transformer 层的数量。           
heads: 多头注意力的头数。                 
mlp_dim: MLP 层的中间维度。                  
pool: 表示在输出时是使用 cls token 还是 mean pooling。                        
channels: 输入图像的通道数，默认为 3（RGB）。             
dim_head: 每个注意力头的维度。                
dropout: Transformer 中 dropout 概率。             
emb_dropout: 位置嵌入后的 dropout。                        

#### 3.1.1 Patch Embedding 部分

```python
image_height, image_width = pair(image_size)
patch_height, patch_width = pair(patch_size)

assert image_height % patch_height == 0 and image_width % patch_width == 0, ...
```

将 image_size 和 patch_size 转为二元组 (height, width)，同时确保图像能被 patch 大小整除。

```python
num_patches = (image_height // patch_height) * (image_width // patch_width)
patch_dim = channels * patch_height * patch_width

```

计算：
- num_patches: 图像总共被划分成多少个 patch。
- patch_dim: 每个 patch 被展平后的维度。

```python
self.to_patch_embedding = nn.Sequential(
    Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_height, p2=patch_width),
    nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    nn.LayerNorm(dim),
)
```

和simple_vit中相同。

### 3.1.2 位置编码和 CLS token

```python
self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
self.dropout = nn.Dropout(emb_dropout)
```

相较于simple_vit，ViT中使用可学习的位置编码（原论文便是这么做的），同时加入了cls_token。
- pos_embedding: 可学习的位置编码（包括 1 个 cls token）。
- cls_token: 类别标识 token，类似于 NLP 中 BERT 的 [CLS]。
- dropout: 对加完位置编码的向量进行 dropout。

### 3.1.3 Transformer encoder-only

```python
self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)
```

就是前面实现的Transformer Encoder。

### 3.1.4 Pooling 与分类头

```python
self.pool = pool
self.to_latent = nn.Identity()
self.mlp_head = nn.Linear(dim, num_classes)
```

这里的self.pool是lucidrains自己加的，原论文中只是用了CLS，并没有用这部分，使用 mean pooling 是将所有 patch token（不包括 cls token）或包括 cls token 取平均作为图像表示，有时候在小数据集或迁移学习场景中会表现的更好一些。
- self.pool: 控制是取 cls_token 还是取所有 token 的平均。
- to_latent 是占位的，保留了进一步处理的可能性（simple_vit也有类似处理）。
- mlp_head: 最终分类头，将 Transformer 的输出映射为 num_classes 个类别。

In [ ]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '1 1 d -> b 1 d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

# 训练 & 测试

这一部分我就不再赘述了，与simple_vit中一样。       
但是后面我可能会继续写一个使用HuggingFace中开源的模型库，跳过训练步骤，直接微调来使用模型。